In [1]:
## headings
"""
made by weiyw @ 2019-03-08
made to use the deep Spatio-temporal Residual Network
"""
# from __future__ import print_function
# from preprocessing import *
# import numpy as np
# np.random.seed(2333)  # for reproducibility


import os
import time
# import h5py
import struct
import numpy as np
from glob import glob

import tensorflow as tf
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

import keras
from keras.models import Model
from keras.layers.convolutional import Convolution2D
# from keras.layers.convolutional import Conv3d
from keras.layers.convolutional import Conv3D
from keras.layers.normalization import BatchNormalization
from keras.layers import (
    Input,
    Activation,
    merge,
    Dense,
    Reshape
)
# from STResNet import stresnet
import metrics as metrics

# from PIL import Image
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
def get_min_nonzero_value(data_in, init_min = 1):
    min_value = init_min
    for bb in data_in:
        for aa in bb:
            if abs(aa) < min_value and aa != 0:
                min_value = abs(aa)
    return min_value

In [3]:
def get_bindata(data_path, data_index, data_phase, nt, nr):
    # data_index start from 1
    model_num = int( ( int(data_index) - 1 ) / 51) + 1
    source_num = ( int(data_index) - 1 ) % 51 + 1
    data_name = data_path + "/model" + str(model_num) + "source" + str(source_num) + data_phase + ".bin"
#     print(data_name)
    
    out_data = np.empty((nt,nr))
    FA = open(data_name, "rb")
    FA.seek(3232,0)
    for tt in range(nt):
        for rr in range(nr):
            data = FA.read(4)
            data_float = struct.unpack("f", data)[0]
            out_data[tt][rr] = data_float
     #np.shape(out_data)   # (4000,301)
#     cut_data = out_data[0:4000, 8:401]
    return out_data  

In [4]:
class Dataloader():
    def __init__(self, data_path, nt, nr, nph):
        self.data_path = data_path
        self.nt = nt
        self.nr = nr
        self.nph = nph
    
    
    def readin_bin_data(self, data_name):
        out_data = np.empty((self.nt, self.nr))
        FA = open(data_name, "rb")
        FA.seek(3232,0)
        for tt in range(self.nt):
            for rr in range(self.nr):
                data = FA.read(4)
                data_float = struct.unpack("f", data)[0]
                out_data[tt][rr] = data_float
         #np.shape(out_data)   # (4000,301)
    #     cut_data = out_data[0:4000, 8:401]
        return out_data  
    
    
#     def load_data(self, batch_size=1, is_testing=False):
#         data_type = "train" if not is_testing else "test"
# #         path = os.path.join(self.data_path, self.dataset_name)
        
#         return x_data, y_data
    
    
    def load_batch(self, batch_size=1, is_testing=False, ratio=0.5):
        data_type = "train" if not is_testing else "test"
#         path = glob('%s/%s/%s/*' % (self.data_path, self.dataset_name, data_type))
        path = glob('%s/vz/*' % self.data_path)
#         path = glob('./datasets/%s/%s/*' % (self.dataset_name, data_type))
#         path = os.path.join(self.data_path, self.dataset_name)
        
        self.n_batches = int( len(path) / batch_size * ratio )
#         self.n_batches = 1
        
#         while True:
        for i in range(self.n_batches):
#             batch = path[ i * batch_size : (i+1) * batch_size]
#             batch_num = [ i * batch_size : (i+1) * batch_size]
#             x_data, y_data = [], [] ## 用list还是array？
            x_data = np.empty((batch_size, self.nt, self.nr, self.nph)) ## b-4000-300-2, vz, vx
            y_data = np.empty((batch_size, self.nt, self.nr, self.nph)) ## b-4000-300-2, div, curl
            for data_index_local in range(batch_size): ## 0 ~ batchsize-1
                data_index = i * batch_size + data_index_local + 1
                model_num = int( ( int(data_index) - 1 ) / 51) + 1
                source_num = ( int(data_index) - 1 ) % 51 + 1
                if self.nph == 2:
                    for data_phase in ['vz', 'vx']:
                        p_flag = int(data_phase == 'vx')
                        data_name = self.data_path + "/" + data_phase + \
                        "/model" + str(model_num) + "source" + str(source_num) + data_phase + ".bin"
#                         print(data_name)
                        x_data[data_index_local, :, :, p_flag] = self.readin_bin_data(data_name)
                    for data_phase in ['div', 'curl']:
                        p_flag = int(data_phase == 'curl')
                        data_name = self.data_path + "/" + data_phase + \
                        "/model" + str(model_num) + "source" + str(source_num) + data_phase + ".bin"
#                         print(data_name)
                        y_data[data_index_local, :, :, p_flag] = self.readin_bin_data(data_name)
                if self.nph == 1:
                    data_phase = 'vz'
                    data_name = self.data_path + "/" + data_phase + \
                    "/model" + str(model_num) + "source" + str(source_num) + data_phase + ".bin"
                    x_data[data_index_local, :, :, 0] = self.readin_bin_data(data_name)
                    data_phase = 'div'
                    data_name = self.data_path + "/" + data_phase + \
                    "/model" + str(model_num) + "source" + str(source_num) + data_phase + ".bin"
                    y_data[data_index_local, :, :, 0] = self.readin_bin_data(data_name)                    
                yield x_data, y_data

In [5]:
def simpleNet(nt, nr, nph):
    x_input = Input( shape=( nt, nr, nph) )##one_piece
    conv1 = Convolution2D(
        nb_filter=64, nb_row=3, nb_col=3, border_mode="same", data_format="channels_last")(x_input)
    conv1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                               beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', 
                               moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, 
                               beta_constraint=None, gamma_constraint=None)(conv1)
    conv2 = Convolution2D(
        nb_filter=2, nb_row=3, nb_col=3, border_mode="same", data_format="channels_last")(conv1)
    # conv1 = Conv3D(filters=51, kernel_size=(2,2,2), strides=(1,1,1), 
    #                             padding='same', data_format="channels_last", dilation_rate=(1,1,1),
    #                            activation=None, use_bias=True, kernel_initializer='glorot_uniform',
    #                            bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None,
    #                            activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(x_input)
    # conv1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
    #                            beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', 
    #                            moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, 
    #                            beta_constraint=None, gamma_constraint=None)(conv1)
    # conv2 = Conv3D(filters=51, kernel_size=(2,2,2), strides=(1,1,1), 
    #                             padding='same', data_format="channels_last", dilation_rate=(1,1,1),
    #                            activation=None, use_bias=True, kernel_initializer='glorot_uniform',
    #                            bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None,
    #                            activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(conv1)

    # conv2 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
    #                            beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', 
    #                            moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, 
    #                            beta_constraint=None, gamma_constraint=None)(conv2)

    # conv3 = Conv3D(filters=51, kernel_size=(2,2,2), strides=(1,1,1), 
    #                             padding='same', data_format="channels_last", dilation_rate=(1,1,1),
    #                            activation=None, use_bias=True, kernel_initializer='glorot_uniform',
    #                            bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None,
    #                            activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(conv2)
    # conv3 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
    #                            beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', 
    #                            moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, 
    #                            beta_constraint=None, gamma_constraint=None)(conv3)

    # convLSTM_1 = keras.layers.ConvLSTM2D(filters=51, kernel_size=(2,2), strides=(1, 1), padding='valid', 
    #                                      data_format='channels_last', dilation_rate=(1, 1), activation='tanh', 
    #                                      recurrent_activation='hard_sigmoid', use_bias=True,
    #                                      kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', 
    #                                      bias_initializer='zeros', unit_forget_bias=True, 
    #                                      kernel_regularizer=None, recurrent_regularizer=None, 
    #                                      bias_regularizer=None, activity_regularizer=None, 
    #                                      kernel_constraint=None, recurrent_constraint=None, 
    #                                      bias_constraint=None, return_sequences=True, 
    #                                      go_backwards=False, stateful=False, dropout=0.0, recurrent_dropout=0.0)(conv3)


    model = Model(inputs=x_input, outputs=conv2)
    # model.summary()
    return model

In [6]:
data_path = "/media/wywdisk/VSPdata/data/haveinvx/layer2_haveinvx"

nt = 4000     # time step
nr = 400      # receiver
ns = 51       # shot
nmodel = 2    # batch
nph = 2

model = simpleNet(nt, nr, nph)
my_data_loader = Dataloader(data_path, nt, nr, nph)

/home/wyw/anaconda3/envs/tf-keras/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(data_format="channels_last", kernel_size=(3, 3), filters=64, padding="same")`
  after removing the cwd from sys.path.
/home/wyw/anaconda3/envs/tf-keras/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(data_format="channels_last", kernel_size=(3, 3), filters=2, padding="same")`
  # Remove the CWD from sys.path while we load stuff.


In [7]:
# ## check for input data
# show_data = one_piece[:,:,0,0]
# # show_data = this_data[0]
# target = "show_data" #"this_data[0]"
# print "The shape of ", target, "\tis", eval(target).shape
# print "The max:\t",  eval(target).max(), "\tThe min:\t",  eval(target).min()
# print "The mean:\t",  eval(target).mean(), "\tThe std:\t",  eval(target).std()
# print "The absolute minimal value is\t", get_min_nonzero_value(eval(target))
# print "==========================================================================="
# show_data = one_piece[:,:,1,0]
# # show_data = this_data[1]
# target = "show_data" #"this_data[0]"
# print "The shape of ", target, "\tis", eval(target).shape
# print "The max:\t",  eval(target).max(), "\tThe min:\t",  eval(target).min()
# print "The mean:\t",  eval(target).mean(), "\tThe std:\t",  eval(target).std()
# print "The absolute minimal value is\t", get_min_nonzero_value(eval(target))

In [8]:
## prepare data for input
## 如何迭代输入数据？
# this_data_3d = one_piece.reshape((nt*nph,nr,ns))
# indata = tf.convert_to_tensor(this_data_3d)

# this_data_5d = one_piece #.reshape((nt*nph,nr,ns))
# indata = tf.convert_to_tensor(this_data_5d)

In [9]:
## training parameters
TPepoches = 4
TPbatch_size = 4

In [10]:
# for epoch in range(TPepoches):
#     for bacth_ii, (x_data, y_data) in \
#     enumerate(my_data_loader.load_batch(batch_size=TPbatch_size, is_testing=False, ratio=0.001)):       

In [16]:
## make single data
ts = time.time()
data_path = "/media/wywdisk/VSPdata/data/haveinvx/layer2_haveinvx"
havebatch = True; data_phases = ["vz", "vx"];  nmodel = 1

if havebatch:
    ## prepare files
    one_piece = np.empty((nmodel,nt, nr, nph)) # 4000, 300, 2, 51
    # model_count = 1    # for iteration [1, totalNum]
    data_index = 1 ## change here !!!
    for data_phase in data_phases:
        p_flag = int(data_phase == 'vx')      # vz=0, vx=1
        one_piece[0,:,:,p_flag] = get_bindata(os.path.join(data_path, data_phase), data_index, data_phase, nt, nr) 

    valid_piece = np.empty((nmodel,nt, nr, nph)) # 4000, 300, 2, 51
    # model_count = 1    # for iteration [1, totalNum]
    data_index = 2
    for data_phase in data_phases:
        p_flag = int(data_phase == 'vx')      # vz=0, vx=1
        valid_piece[0,:,:,p_flag] = get_bindata(os.path.join(data_path, data_phase), data_index, data_phase, nt, nr) 

    test_piece = np.empty((nmodel,nt, nr, nph)) # 4000, 300, 2, 51
    # model_count = 1    # for iteration [1, totalNum]
    data_index = 3
    for data_phase in data_phases:
        p_flag = int(data_phase == 'vx')      # vz=0, vx=1
        test_piece[0,:,:,p_flag] = get_bindata(os.path.join(data_path, data_phase), data_index, data_phase, nt, nr) 

else:    
    ## prepare files
    one_piece = np.empty((nt, nr, nph)) # 4000, 300, 2, 51
    # model_count = 1    # for iteration [1, totalNum]
    data_index = 1
    for data_phase in data_phases:
        p_flag = int(data_phase == 'vx')      # vz=0, vx=1
        one_piece[:,:,p_flag] = get_bindata(os.path.join(data_path, data_phase), data_index, data_phase, nt, nr) 

    valid_piece = np.empty((nt, nr, nph)) # 4000, 300, 2, 51
    # model_count = 1    # for iteration [1, totalNum]
    data_index = 2
    for data_phase in data_phases:
        p_flag = int(data_phase == 'vx')      # vz=0, vx=1
        valid_piece[:,:,p_flag] = get_bindata(os.path.join(data_path, data_phase), data_index, data_phase, nt, nr) 

    test_piece = np.empty((nt, nr, nph)) # 4000, 300, 2, 51
    # model_count = 1    # for iteration [1, totalNum]
    data_index = 3
    for data_phase in data_phases:
        p_flag = int(data_phase == 'vx')      # vz=0, vx=1
        test_piece[:,:,p_flag] = get_bindata(os.path.join(data_path, data_phase), data_index, data_phase, nt, nr) 


if havebatch :
    ## prepare files
    one_piece_y = np.empty((nmodel,nt, nr, nph)) # 4000, 300, 2, 51
    # model_count = 1    # for iteration [1, totalNum]
    data_index = 1
    for data_phase in data_phases:
        p_flag = int(data_phase == 'curl')      # vz=0, vx=1
        one_piece_y[0,:,:,p_flag] = get_bindata(os.path.join(data_path, data_phase), data_index, data_phase, nt, nr) 

    ## prepare files
    valid_piece_y = np.empty((nmodel,nt, nr, nph)) # 4000, 300, 2, 51
    # model_count = 1    # for iteration [1, totalNum]
    data_index = 2
    for data_phase in data_phases:
        p_flag = int(data_phase == 'curl')      # vz=0, vx=1
        valid_piece_y[0,:,:,p_flag] = get_bindata(os.path.join(data_path, data_phase), data_index, data_phase, nt, nr) 

    ## prepare files
    test_piece_y = np.empty((nmodel,nt, nr, nph)) # 4000, 300, 2, 51
    # model_count = 10    # for iteration [1, totalNum]
    data_index = 3
    for data_phase in data_phases:
        p_flag = int(data_phase == 'curl')      # vz=0, vx=1
        test_piece_y[0,:,:,p_flag] = get_bindata(os.path.join(data_path, data_phase), data_index, data_phase, nt, nr) 
else:
    ## prepare files
    one_piece_y = np.empty((nt, nr, nph)) # 4000, 300, 2, 51
    # model_count = 1    # for iteration [1, totalNum]
    data_index = 1
    for data_phase in data_phases:
        p_flag = int(data_phase == 'curl')      # vz=0, vx=1
        one_piece_y[:,:,p_flag] = get_bindata(os.path.join(data_path, data_phase), data_index, data_phase, nt, nr) 

    ## prepare files
    valid_piece_y = np.empty((nt, nr, nph)) # 4000, 300, 2, 51
    # model_count = 1    # for iteration [1, totalNum]
    data_index = 2
    for data_phase in data_phases:
        p_flag = int(data_phase == 'curl')      # vz=0, vx=1
        valid_piece_y[:,:,p_flag] = get_bindata(os.path.join(data_path, data_phase), data_index, data_phase, nt, nr) 

    ## prepare files
    test_piece_y = np.empty((nt, nr, nph)) # 4000, 300, 2, 51
    # model_count = 10    # for iteration [1, totalNum]
    data_index = 3
    for data_phase in data_phases:
        p_flag = int(data_phase == 'curl')      # vz=0, vx=1
        test_piece_y[:,:,p_flag] = get_bindata(os.path.join(data_path, data_phase), data_index, data_phase, nt, nr) 
        
# print("\nelapsed time (compiling model): %.3f seconds\n" %
#         (time.time() - ts))

# x_train, y_train = tf.convert_to_tensor(one_piece), tf.convert_to_tensor(one_piece_y)
x_valid, y_valid = tf.convert_to_tensor(valid_piece), tf.convert_to_tensor(valid_piece_y)
x_test, y_test = tf.convert_to_tensor(test_piece), tf.convert_to_tensor(test_piece_y)

# print(x_train.shape)
# print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)
print(x_test.shape)
print(y_test.shape)
print("\nelapsed time (compiling model): %.3f seconds\n" %
        (time.time() - ts)) 

(1, 4000, 400, 2)
(1, 4000, 400, 2)
(1, 4000, 400, 2)
(1, 4000, 400, 2)

elapsed time (compiling model): 10.214 seconds



In [17]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [18]:
## complete the training process
# adam = Adam(lr=lr)
# model.compile(loss='mse', optimizer=adam, metrics=[rmse])
#  #   model.summary()
model.compile(loss='mean_squared_error',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
## https://keras.io/models/model/
# '''
# loss: mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, mean_squared_logarithmic_error,
#         squared_hinge, categorical_hinge, logcosh, categorical_crossentropy, sparse_categorical_crossentropy,
#         binary_crossentropy, kullback_leibler_divergence, poisson, cosine_proximity
# optimizer: SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam
# metrics: ?
# '''

Learning rate:  0.001


In [19]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
train_size = 1
test_size = 1
time_size = 4000
nb_residual_unit = 0
lr = 0
# model_name = 'vsp_%s_model.{epoch:03d}.h5' % model_type
hyperparams_name = 'train_size{}.test_size{}.time_size{}.resunit{}.lr{}'.format(
        train_size, test_size, time_size, nb_residual_unit, lr)
fname_param = os.path.join(save_dir,'{}.best.h5'.format(hyperparams_name))
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = fname_param#os.path.join(save_dir, fname_param)

# early_stopping = EarlyStopping(monitor='val_rmse', patience=2, mode='min')
# model_checkpoint = ModelCheckpoint(
#     fname_param, monitor='rmse', verbose=0, save_best_only=True, mode='min')

# print("\nelapsed time (compiling model): %.3f seconds\n" %
#         (time.time() - ts))

# print('=' * 10)

checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=0,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)
# callbacks = [checkpoint, lr_reducer, lr_scheduler]
callbacks = [checkpoint]#, lr_reducer, lr_scheduler]

In [20]:
# model.load_weights(os.path.join('MODEL', '{}.best.h5'.format(hyperparams_name)))

In [ ]:
# for bacth_ii, (x_data, y_data) in enumerate(my_data_loader.load_batch(batch_size=TPbatch_size, is_testing=False, ratio=0.001)):
# model.fit_generator(my_data_loader.load_batch(batch_size=TPbatch_size, is_testing=False, ratio=0.001),\
#               steps_per_epoch=None, epochs=1, verbose=1, callbacks=None, validation_data=None, \
#               validation_steps=None, validation_freq=1, class_weight=None, max_queue_size=10, \
#               workers=1, use_multiprocessing=False, shuffle=False, initial_epoch=0)

log = model.fit_generator(my_data_loader.load_batch(batch_size=TPbatch_size, is_testing=False, ratio=0.1),\
              steps_per_epoch=TPbatch_size, epochs=TPepoches, verbose=1, callbacks=callbacks, validation_data=None, \
              validation_steps=None, class_weight=None, max_queue_size=10, \
              workers=1, use_multiprocessing=False, shuffle=False, initial_epoch=0)



Epoch 1/4
4/4 [==============================] - 22s 5s/step - loss: 9.5217e-12 - acc: 0.5147


/home/wyw/anaconda3/envs/tf-keras/lib/python3.6/site-packages/keras/callbacks.py:434: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


Epoch 2/4
4/4 [==============================] - 18s 4s/step - loss: 3.1291e-06 - acc: 0.5003
Epoch 3/4
4/4 [==============================] - 18s 5s/step - loss: 2.2009e-06 - acc: 0.6035
Epoch 4/4
3/4 [=====================>........] - ETA: 4s - loss: 8.2012e-07 - acc: 0.3336

In [ ]:
# # print "training model..."
# ts = time.time()

# # model.fit(x_train, y_train,
# #           batch_size=batch_size,
# #           epochs=epochs,
# #           validation_data=(x_valid, y_valid),
# #           shuffle=True,
# #           steps_per_epoch=None,
# #           validation_steps=None,
# #           callbacks=callbacks)
# model.fit(x_train, y_train,
#           batch_size=None,
#           epochs=epochs,
#           validation_data=(x_valid, y_valid),
#           shuffle=True,
#           steps_per_epoch=1,
#           validation_steps=1,
#           callbacks=callbacks)
# print("\nelapsed time (training cont): %.3f seconds\n" % (time.time() - ts))

In [ ]:
# model.load_weights(fname_param)
# from models import model_from_json

json_string = model.to_json()
open('model_json', 'w').write(json_string)
#model = model_from_json(json_string)
model.save_weights(os.path.join( save_dir, '{}.final.best.h5'.format(hyperparams_name)), overwrite=True)

In [ ]:
# Score trained model.
ts = time.time()
scores = model.evaluate(x_valid, y_valid, batch_size=None, steps=1, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# print('Test score: %.6f rmse (norm): %.6f rmse (real): %.6f' %
#         (score[0], score[1], score[1] * (mx - mn) / 2.))
print("\nelapsed time (eval cont): %.3f seconds\n" % (time.time() - ts))

In [ ]:
## prdiction
# y_predict = model.predict(x_test, batch_size=None, steps=1, verbose = 1)

predict = model.predict_generator(my_data_loader.load_batch(batch_size=TPbatch_size, is_testing=True, ratio=0.1),\
                                  steps=1, max_queue_size=10, \
                                  workers=1, use_multiprocessing=False, verbose=0)

In [ ]:
## check for predicted data
a1 = y_predict
print("Information of a1 :")
print("The shape of a1   :",a1.shape)
print("The range        : ", [a1.min(),a1.max()])

In [ ]:
## plot y_predict
from matplotlib.ticker import MultipleLocator
lines = 1
cols = 2
forts = 12
ss1 = 300
ss2 = 400

x_libs = [0,0, 500, 1000, 1500, 2000]#np.linspace(0,1500,4)#[0, 500, 1000, 1500]
y_libs = np.linspace(0,2,9)#[0,0.4, 0.8, 1.2, 1.6, 2]

fig, axs = plt.subplots(1,2,figsize=(10, 4), sharey=True)

# obj = 'y_predict'
# im = plt.imshow(eval(obj)[0,:,:,1], extent=[0, 512, 512,0])
# plt.colorbar(im)

# ax1 = plt.subplot(131)
ax1 = plt.subplot(lines,cols,1)
obj = 'y_predict'
# print(obj, ": ", eval(obj).shape, "\t", [eval(obj).min(), eval(obj).max()] )
im = plt.imshow(eval(obj)[0,:,:,0], extent=[0, ss1, ss2,0])
plt.colorbar(im)
# plt.title("The elastic z component of VSP data", fontsize=forts);
plt.title("predicted P-wave", fontsize=forts);
plt.xlabel('depth(m)', fontsize=forts);
plt.ylabel('time(s)', fontsize=forts);
# ax1 = fig.add_subplot(131)
ax1.xaxis.set_major_locator(MultipleLocator(75))
ax1.xaxis.set(ticklabels=x_libs)#[0,300,600, 900, 1200, 1500]);
ax1.yaxis.set(ticklabels=y_libs)#[0,0.4, 0.8, 1.2, 1.6, 2]);
plt.setp(ax1.get_xticklabels(), fontsize=10);
# plt.tight_layout()

# ax1 = plt.subplot(131)
ax2 = plt.subplot(lines,cols,2)
obj = 'y_predict'
# print(obj, ": ", eval(obj).shape, "\t", [eval(obj).min(), eval(obj).max()] )
im = plt.imshow(eval(obj)[0,:,:,1], extent=[0, ss1, ss2,0])
plt.colorbar(im)
# plt.title("The elastic z component of VSP data", fontsize=forts);
plt.title("predicted S-wave", fontsize=forts);
plt.xlabel('depth(m)', fontsize=forts);
plt.ylabel('time(s)', fontsize=forts);
# ax1 = fig.add_subplot(131)
ax2.xaxis.set_major_locator(MultipleLocator(75))
ax2.xaxis.set(ticklabels=x_libs)#[0,300,600, 900, 1200, 1500]);
ax2.yaxis.set(ticklabels=y_libs)#[0,0.4, 0.8, 1.2, 1.6, 2]);
plt.setp(ax2.get_xticklabels(), fontsize=10);
# plt.tight_layout()

